In [107]:
using Pkg
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using DataFrames

  Activating project at `~/Documents/MacroEnergy.jl`


In [128]:
path = "/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/eastern_us_three_zones_with_cement/assets/cement_retrofit.json"
data = MacroEnergy.load_inputs(path)[:TradCementPlant][1]

data[:global_data][:edges][:cement_edge][:retrofit_options]

Dict{Symbol, Any} with 1 entry:
  :oxyfuel => Dict{Symbol, Any}(:variable_om_cost=>0, :fixed_om_cost=>58, :is_r…

In [129]:
function find_retrofit_paths(d::AbstractDict, path=[])
    found_paths = []
    for (k, v) in d
        new_path = push!(copy(path), k)
        if k == :retrofit_options
            push!(found_paths, new_path)
        elseif isa(v, AbstractDict)
            append!(found_paths, find_retrofit_paths(v, new_path))
        end
    end
    return found_paths
end

find_retrofit_paths (generic function with 2 methods)

In [130]:
find_retrofit_paths(data)

2-element Vector{Any}:
 Any[:global_data, :edges, :fuel_edge, :retrofit_options]
 Any[:global_data, :edges, :cement_edge, :retrofit_options]

In [ ]:
path = @__DIR__
system = MacroEnergy.load_system(case_path)

model = MacroEnergy.generate_model(system)

MacroEnergy.set_optimizer(model, Gurobi.Optimizer);

MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)

MacroEnergy.optimize!(model)

In [ ]:
system = MacroEnergy.empty_system(dirname(path))
system_data = MacroEnergy.load_system_data(joinpath(path, "system_data.json"))
data = system_data[:assets]

┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25


Dict{Symbol, Any} with 1 entry:
  :path => "assets"

In [ ]:
data = MacroEnergy.load_inputs("assets/tradcementplant.json")[:TradCementPlant][1]

instance_data = MacroEnergy.merge_global_data(data)[1]
instance_data[:instance_data]

Dict{Symbol, Any} with 7 entries:
  :location          => "MA"
  :id                => "traditional_cement_MA"
  :existing_capacity => 100
  :fuel_commodity    => "NaturalGas"
  :nodes             => Dict{Symbol, Any}()
  :edges             => Dict{Symbol, Any}(:elec_edge=>Dict{Symbol, Any}(:type=>…
  :transforms        => Dict{Symbol, Any}(:constraints=>Dict{Symbol, Any}(:Bala…

In [106]:
data[:global_data]

Dict{Symbol, Any} with 4 entries:
  :fuel_commodity => "NaturalGas"
  :nodes          => Dict{Symbol, Any}()
  :edges          => Dict{Symbol, Any}(:elec_edge=>Dict{Symbol, Any}(:type=>"El…
  :transforms     => Dict{Symbol, Any}(:constraints=>Dict{Symbol, Any}(:Balance…

In [99]:
MacroEnergy.check_and_convert_type(instance_data)

ThermalPower